In [91]:
import paramiko
import getpass

host = "158.251.88.197"
port = 22
username = "iaraya"
password = getpass.getpass()

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

 ········


In [107]:
import random

def write_instance(L,W,H,boxes,filename):
    txt = "1\n1 0\n"+str(L)+" "+str(W)+" "+str(H)+"\n";
    txt += str(len(boxes)) + "\n"
    for box in boxes:
        txt += str(box.id) + ' ' + str(box.l) + ' ' + str(box.rotx) \
            + ' ' + str(box.w) + ' ' + str(box.roty) \
            + ' ' + str(box.h) + ' ' + str(box.rotz) + ' ' + str(boxes[box])  + '\n'
    txt += '\n'

    text_file = open(filename, "w")
    text_file.write(txt)
    text_file.close()


In [129]:
import os
def bsg_solve(L,W,H,boxes,id2box, time=1):
    filename="tmp_instance_"+ str(random.randint(10000, 99999))
    
    write_instance(L,W,H,boxes,filename)
    from scp import SCPClient
    scp = SCPClient(ssh.get_transport())
    scp.put(filename, "/home/iaraya/clp/"+filename)

    stdin, stdout, stderr = ssh.exec_command("/home/iaraya/clp/BSG_CLP /home/iaraya/clp/"+filename+" -i 0 -t "+str(time)+" --json")
    lines = stdout.readlines()
    print(lines[-1])
    json_data = json.loads(lines[-1])
    
    loaded={}; remaining={}
    for item in json_data["loaded"]:
        loaded[id2box[item[0]]]=item[1]
    for item in json_data["remaining"]:
        remaining[id2box[item[0]]]=item[1]
        
    ssh.exec_command("rm /home/iaraya/clp/"+filename)

    os.remove(filename)
    return loaded, remaining, json_data["utilization"]


In [130]:
def load_BRinstance(filename, inst=1):
    boxes={}; id2box={}
    with open(filename) as f:
        n = int(next(f))
        for i in range(n):
            next(f)
            L, W, H =  [int(x) for x in next(f).split()]
            n_boxes = int(next(f))
            for it in range(n_boxes):
                id, l, rotx, w, roty, h, rotz, nn = [int(x) for x in next(f).split()] 
                if i == inst-1:
                    b = box(id, l, w, h, rotx, roty, rotz)
                    boxes[b]=nn;
                    id2box[id]=b
            if i == inst-1: return L,W,H,boxes,id2box

### Ejemplo usando instancia BR

In [131]:
L,W,H,boxes,id2box = load_BRinstance("BR8.txt",3)
remaining, loaded, utilization = bsg_solve(L,W,H,boxes,id2box,time=5)

{"remaining" :[[5,1],[7,1],[9,1],[20,2],[25,1],[28,1],[29,1]], "loaded" :[[1,2],[2,4],[3,5],[4,5],[5,6],[6,4],[7,6],[8,3],[9,4],[10,3],[11,6],[12,4],[13,5],[14,4],[15,2],[16,3],[17,2],[18,7],[19,4],[20,2],[21,2],[22,3],[23,7],[24,6],[25,9],[26,4],[27,4],[28,3],[29,3],[30,7]], "utilization" : 0.94444486}



### Ejemplo creando instancia y resolviendo

In [132]:
boxes={}
b=box(1,10,10,10,1,1,1)
boxes[b]=5; id2box[1]=b # 5 cajas de 10x10x10 
b=box(2,20,20,20,1,1,1)
boxes[b]=5; id2box[2]=b # 5 cajas de 20x20x20 
b=box(3,40,40,40,1,1,1)
boxes[b]=5; id2box[3]=b # 5 cajas de 40x40x40 
b=box(4,80,80,80,1,1,1)
boxes[b]=5; id2box[4]=b # 5 cajas de 80x80x80 

remaining, loaded, utilization = bsg_solve(100,100,100,boxes,id2box, time=1)

{"remaining" :[[3,5],[4,4]], "loaded" :[[1,5],[2,5],[4,1]], "utilization" : 0.557}

